## Part 2: Training your own ML Model

<a href="https://colab.research.google.com/github/peckjon/hosting-ml-as-microservice/blob/master/part2/train_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download corpuses

We'll continue using the `movie_reviews` corpus to train our model. The `stopwords` corpus contains a [set of standard stopwords](https://gist.github.com/sebleier/554280) we'll want to remove from the input, and `punkt` is used for toneization in the [.words()](https://www.nltk.org/api/nltk.corpus.html#corpus-reader-functions) method of the corpus reader.

In [1]:
from nltk import download

download('movie_reviews')
download('punkt')
download('stopwords')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\kandp\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kandp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kandp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Define feature extractor and bag-of-words converter

Given a list of (already tokenized) words, we need a function to extract just the ones we care about: those not found in the list of English stopwords or standard punctuation.

We also need a way to easily turn a list of words into a [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model), pairing each word with the count of its occurrences.

In [2]:
from nltk.corpus import stopwords
from string import punctuation

stopwords_eng = stopwords.words('english')

def extract_features(words):
    return [w for w in words if w not in stopwords_eng and w not in punctuation]

def bag_of_words(words):
    bag = {}
    for w in words:
        bag[w] = bag.get(w,0)+1
    return bag

### Ingest, clean, and convert the positive and negative reviews

For both the positive ("pos") and negative ("neg") sets of reviews, extract the features and convert to bag of words. From these, we construct a list of tuples known as a "featureset": the first part of each tuple is the bag of words for that review, and the second is its label ("pos"/"neg").

Note that `movie_reviews.words(fileid)` provides a tokenized list of words. If we wanted the un-tokenized text, we would use `movie_reviews.raw(fileid)` instead, then tokenize it using our preferred tokenizeer (e.g. [nltk.tokenize.word_tokenize](https://www.nltk.org/api/nltk.tokenize.html#nltk.tokenize.punkt.PunktLanguageVars.word_tokenize)).

In [3]:
from nltk.corpus import movie_reviews

reviews_pos = []
reviews_neg = []
for fileid in movie_reviews.fileids('pos'):
    words = extract_features(movie_reviews.words(fileid))
    reviews_pos.append((bag_of_words(words), 'pos'))
for fileid in movie_reviews.fileids('neg'):
    words = extract_features(movie_reviews.words(fileid))
    reviews_neg.append((bag_of_words(words), 'neg'))

### Split reviews into training and test sets
We need to break up each group of reviews into a training set (about 80%) and a test set (the remaining 20%). In case there's some meaningful order to the reviews (e.g. the first 800 are from one group of reviewers, the next 200 are from another), we shuffle the sets first to ensure we aren't introducing additional bias. Note that this means our accuracy will not be exactly the same on every run; if you wish to see consistent results on each run, you can stabilize the shuffle by calling [random.seed(n)](https://www.geeksforgeeks.org/random-seed-in-python/) first.

In [4]:

# def split_idx(size, split_pct):
#     train_size = int(size*split_pct)
#     total_idx = list(range(size))
#     train_idx = random.sample(total_idx, train_size)
#     test_idx = list(set(total_idx).difference(set(train_idx)))
#     return (train_idx, test_idx)

# train_idx, test_idx = split_idx(len(reviews_pos), 0.8)

In [5]:
# pos_train, pos_test = reviews_pos[train_idx], reviews_pos[test_idx]
# neg_train, neg_tes = reviews_neg[train_idx], reviews_neg[test_idx]

In [6]:
import random
from random import shuffle

In [7]:
random.seed(42)

In [8]:


split_pct = .80

def split_set(review_set):
    split = int(len(review_set)*split_pct)
    return (review_set[:split], review_set[split:])

shuffle(reviews_pos)
shuffle(reviews_neg)

pos_train, pos_test = split_set(reviews_pos)
neg_train, neg_test = split_set(reviews_neg)

train_set = pos_train+neg_train
test_set = pos_test+neg_test

### Train the model

Now that our data is ready, the training step itself is quite simple if we use the [NaiveBayesClassifier](https://www.nltk.org/api/nltk.classify.html#module-nltk.classify.naivebayes) provided by NLTK.

If you are used to methods such as `model.fit(x,y)` which take two parameters -- the data and the labels -- it may be confusing that `NaiveBayesClassifier.train` takes just one argument. This is because the labels are already embedded in `train_set`: each element in the set is a Bag of Words paired with a 'pos' or 'neg'; value.

In [9]:
from nltk.classify import NaiveBayesClassifier

model = NaiveBayesClassifier.train(train_set)

### Check model accuracy

NLTK's built-in [accuracy](https://www.nltk.org/api/nltk.classify.html#module-nltk.classify.util) utility can run our test_set through the model and compare the labels returned by the model to the labels in the test set, producing an overall % accuracy. Not too impressive, right? We need to improve.

In [10]:
from nltk.classify.util import accuracy

print(100 * accuracy(model, test_set))

74.75


### Save the model
Our trained model will be cleared from memory when this notebook is closed. So that we can use it again later, save the model as a file using the [pickle](https://docs.python.org/3/library/pickle.html) serializer.

In [11]:
import pickle

model_file = open('sa_classifier.pickle','wb')
pickle.dump(model, model_file)
model_file.close()

### Improving Baseline

let us try improving the baseline accuracy further

### Text processing utils

The following functions encapsulate some operations that we will repeat on multiple implementations. Extracting them to independent functions allows us to reduce unnecessary duplication and increase code readability

In [12]:
import re

def is_useful_word(word):
    return (word not in stopwords_eng) and (word not in punctuation)

def remove_punctuation(text):
    return re.sub(r'[^a-zA-Z0-9\s]', ' ', text)

### NLTK Everygram

In [13]:
# IMPLEMENTATION: use n-grams

import re
# from nltk.util import ngrams
from nltk.util import everygrams

def extract_features_evgram(document):
    document = document.lower()
    document = re.sub(r'[^a-zA-Z0-9\s]', ' ', document)
    words = [w for w in document.split(" ") if w != "" and is_useful_word(w)]
    return ['_'.join(ngram) for ngram in list(everygrams(words, max_len=3))]

# Example
print(extract_features_evgram("Hello world, corpuses calling!"))

['hello', 'hello_world', 'hello_world_corpuses', 'world', 'world_corpuses', 'world_corpuses_calling', 'corpuses', 'corpuses_calling', 'calling']


### Ingest, clean, and convert the positive and negative reviews

use everygram on the same process as before 

In [14]:
reviews_pos_evg = []
reviews_neg_evg = []
for fileid in movie_reviews.fileids('pos'):
    words = extract_features_evgram(movie_reviews.raw(fileid))
    reviews_pos_evg.append((bag_of_words(words), 'pos'))
for fileid in movie_reviews.fileids('neg'):
    words = extract_features_evgram(movie_reviews.raw(fileid))
    reviews_neg_evg.append((bag_of_words(words), 'neg'))

### Split reviews into training and test sets

In [15]:
from random import shuffle

shuffle(reviews_pos_evg)
shuffle(reviews_neg_evg)

pos_train, pos_test = split_set(reviews_pos_evg)
neg_train, neg_test = split_set(reviews_neg_evg)

train_set = pos_train+neg_train
test_set = pos_test+neg_test

### Train the model

Retraining the NaiveBayesClassifier model on the new data 

In [16]:
model_nb_evg = NaiveBayesClassifier.train(train_set)

### Check model accuracy

NLTK's built-in [accuracy](https://www.nltk.org/api/nltk.classify.html#module-nltk.classify.util) utility can run our test_set through the model and compare the labels returned by the model to the labels in the test set, producing an overall % accuracy. Not too impressive, right? We need to improve.

In [17]:
print(100 * accuracy(model_nb_evg, test_set))

80.5


### Save the model
Our trained model will be cleared from memory when this notebook is closed. So that we can use it again later, save the model as a file using the [pickle](https://docs.python.org/3/library/pickle.html) serializer.

In [18]:
model_file = open('sa_classifier_evg.pickle','wb')
pickle.dump(model, model_file)
model_file.close()

### Spacy lemmatizer

In [19]:
!pip install spacy
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [20]:
# IMPLEMENTATION: use Spacy lemmatizer

import spacy 

nlp = spacy.load("en_core_web_sm") 

def extract_features_lemm(document):
    return [str(w.lemma_) for w in nlp(document) if is_useful_word(w.text)]

# Example:
print(extract_features_lemm("Hello world, corpuses calling!"))

['hello', 'world', 'corpus', 'call']


### Ingest, clean, and convert the positive and negative reviews

use lemmatized data on the same process as before 

In [21]:
reviews_pos_lemm = []
reviews_neg_lemm = []
for fileid in movie_reviews.fileids('pos'):
    words = extract_features_lemm(movie_reviews.raw(fileid))
    reviews_pos_lemm.append((bag_of_words(words), 'pos'))
for fileid in movie_reviews.fileids('neg'):
    words = extract_features_lemm(movie_reviews.raw(fileid))
    reviews_neg_lemm.append((bag_of_words(words), 'neg'))

### Split reviews into training and test sets

In [22]:
shuffle(reviews_pos_lemm)
shuffle(reviews_neg_lemm)

pos_train, pos_test = split_set(reviews_pos_lemm)
neg_train, neg_test = split_set(reviews_neg_lemm)

train_set = pos_train+neg_train
test_set = pos_test+neg_test

### Train the model

Retraining the NaiveBayesClassifier model on the new data 

In [23]:
model_nb_lemm = NaiveBayesClassifier.train(train_set)

### Check model accuracy

NLTK's built-in [accuracy](https://www.nltk.org/api/nltk.classify.html#module-nltk.classify.util) utility can run our test_set through the model and compare the labels returned by the model to the labels in the test set, producing an overall % accuracy. Not too impressive, right? We need to improve.

In [24]:
from nltk.classify.util import accuracy

print(100 * accuracy(model_nb_lemm, test_set))

66.0


### Save the model
Our trained model will be cleared from memory when this notebook is closed. So that we can use it again later, save the model as a file using the [pickle](https://docs.python.org/3/library/pickle.html) serializer.

In [25]:
import pickle

model_file = open('sa_classifier_lemm.pickle','wb')
pickle.dump(model, model_file)
model_file.close()

### Save the model (Colab version)

Google Colab doesn't provide direct access to files saved during a notebook session, so we need to save it in [Google Drive](https://drive.google.com) instead. The first time you run this, it will ask for permission to access your Google Drive. Follow the instructions, then wait a few minutes and look for a new folder called "Colab Output" in [Drive](https://drive.google.com). Note that Colab does not alway sync to Drive immediately, so check the file update times and re-run this cell if it doesn't look like you have the most revent version of your file.

In [26]:
# import sys
# if 'google.colab' in sys.modules:
#     from google.colab import drive
#     drive.mount('/content/gdrive')
#     !mkdir -p '/content/gdrive/My Drive/Colab Output'
#     model_file = open('/content/gdrive/My Drive/Colab Output/sa_classifier.pickle','wb')
#     pickle.dump(model, model_file)
#     model_file.flush()
#     print('Model saved in /content/gdrive/My Drive/Colab Output')
#     !ls '/content/gdrive/My Drive/Colab Output'
#     drive.flush_and_unmount()
#     print('Re-run this cell if you cannot find it in https://drive.google.com')